In [97]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from catboost import CatBoostClassifier

In [2]:
from nlp_pipeline import *

In [78]:
# 21: LogReg Fasttext
oof21 = pd.read_csv("submissions\\oof_train21.csv")
sub21 = pd.read_csv("submissions\\submission21.csv")
# 22: NB Fasttext
# oof22 = pd.read_csv("submissions\\oof_train22.csv")
# sub22 = pd.read_csv("submissions\\submission22.csv")
# ooflstm4 = pd.read_csv("submissions\\lstm_ft_oof_template.csv")
# sublstm4 = pd.read_csv("submissions\\lstm4.csv")
# 29: LGBM Fasttext
oof29 = pd.read_csv("submissions\\oof_train29.csv")
sub29 = pd.read_csv("submissions\\submission29.csv")
# 31: LogReg char n-grams
oof31 = pd.read_csv("submissions\\oof_train31.csv")
sub31 = pd.read_csv("submissions\\submission31.csv")
# 33: LGBM char n-grams
oof33 = pd.read_csv("submissions\\oof_train33.csv")
sub33 = pd.read_csv("submissions\\submission33.csv")
# 35: NB char n-grams
oof35 = pd.read_csv("submissions\\oof_train35.csv")
sub35 = pd.read_csv("submissions\\submission35.csv")
# 37: LGBM Fasttext
oof37 = pd.read_csv("submissions\\oof_train37.csv")
sub37 = pd.read_csv("submissions\\submission37.csv")
# 39: KNN32
# oof39 = pd.read_csv("submissions\\oof_train39.csv")
# sub39 = pd.read_csv("submissions\\submission39.csv")
# 40: LGMB stacker
oof40 = pd.read_csv("submissions\\oof_train40.csv")
sub40 = pd.read_csv("submissions\\submission40.csv")
# 41: LGBM GloVe
# oof41 = pd.read_csv("submissions\\oof_train41.csv")
# sub41 = pd.read_csv("submissions\\submission41.csv")
# 42: QDA GloVe
# oof42 = pd.read_csv("submissions\\oof_train42.csv")
# sub42 = pd.read_csv("submissions\\submission42.csv")

In [4]:
train = pd.read_csv('data\\train.csv').fillna(' ')
test = pd.read_csv('data\\test.csv').fillna(' ')

In [5]:
labels = [c for c in oof21.columns][1:8]

In [6]:
pretrained = "data\\crawl-300d-2M.vec"
feature_funcs = [len, asterix_freq, uppercase_freq, line_change_freq, rep_freq, question_freq]
transforms = [tokenize]
logreg = LogisticRegression(C=0.2, class_weight='balanced', solver='newton-cg', max_iter=10)
logreg.name = "Logistic regression stacker"
gbm = lgb.LGBMClassifier(max_depth=3, metric="auc", n_estimators=125, num_leaves=10, boosting_type="gbdt", learning_rate=0.1, feature_fraction=0.9, bagging_fraction=0.8, bagging_freq=5, reg_lambda=0.2)
gbm.name = "LightGBM stacker"
models = [gbm]

In [98]:
cb = CatBoostClassifier()

In [104]:
# pipe.models = [cb]
pipe.models = [gbm]

In [7]:
pipe = NlpPipeline(train, test, "comment_text", labels, feature_funcs, transforms, models, word_index=None, pretrained=pretrained)

In [91]:
pipe.engineer_features()

Engineering features


In [92]:
train_meta = np.hstack([np.array(oof21[labels]), np.array(oof31[labels]), np.array(oof33[labels]), np.array(oof35[labels]), np.array(oof37[labels])])
test_meta = np.hstack([np.array(sub21[labels]), np.array(sub31[labels]), np.array(sub33[labels]), np.array(sub35[labels]), np.array(sub37[labels])])

In [93]:
pipe.train_features = np.hstack([pipe.train_features, train_meta])
pipe.test_features = np.hstack([pipe.test_features, test_meta])

In [105]:
pipe.cross_val()

Cross-validating
LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
        class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
        learning_rate=0.1, max_depth=3, metric='auc', min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=125,
        n_jobs=-1, num_leaves=10, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.2, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1)
Cross-validating toxic
roc_auc: 0.984410078723
Cross-validating severe_toxic


KeyboardInterrupt: 

In [96]:
pipe.cv_scores # 0.98898509992700967

{'LightGBM stacker': 0.9889249967220799}

In [18]:
pipe.fit_predict_oof()

Creating out-of-fold meta training set for stacker
LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
        class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
        learning_rate=0.1, max_depth=3, metric='auc', min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=10, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.2, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1)
toxic
AUC: 0.984025009484
AUC: 0.98266670157
AUC: 0.985391553051
AUC: 0.984129414374
AUC: 0.98423642498
severe_toxic
AUC: 0.990964164299
AUC: 0.990755160042
AUC: 0.991195626885
AUC: 0.991339670006
AUC: 0.990610698807
obscene
AUC: 0.994141814531
AUC: 0.992862841341
AUC: 0.994471447949
AUC: 0.992909863777
AUC: 0.99384433693
threat
AUC: 0.994375770828
AUC: 0.992251186141
AUC: 0.991394882339
AUC: 0.988359007974
AUC: 0.981100823958
insult
AUC: 0.986979636
AUC: 0.9874424

In [20]:
pipe.create_submission()

Creating submissions
